In [7]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from utilities import *
from dissipative_utils import *

This notebook contains code to visualize MNO predictions on the 2D Navier-Stokes equations and compare them to baseline models. While the code is written specifically for visualizing predictions on the 2D Navier-Stokes equations, the notebook can easily be adapted to visualizing predictions on other infinite-dimensional PDE systems.

# Setup

In [5]:
# Load Data
data = np.load('/central/groups/tensorlab/zongyi/data/KFvorticity_Re500_N1000_T500.npy')

res = data.shape[-1] # for simplicity, assume x and y resolutions are equal
num_traj = 25 # number of trajectories for sampling
T_in = 100 # > 0 to allow time for generated trajectories to reach attractor
T_out = 500

data = data[:num_traj, T_in:T_out, :, :].reshape(-1, res, res)

truth = torch.tensor(data, dtype=torch.float)

MemoryError: Unable to allocate 15.3 GiB for an array with shape (2052096000,) and data type float64

In [ ]:
# Model setup
Re = 500
model_path = 'MODEL_PATH'
model = ''

To compare the learned invariant statistics of our models with the true invariant statistics of the system, we need an accurate representation of the models' learned attractors. Thus, we need a long (e.g., 10,000 time-steps) trajectory generated from the model's predictions. Since the model is Markovian, we can give it an initial condition and repeatedly compose it with itself.

Since we are considering ergodic systems, the initial condition does not matter. We take it to be some arbitrary point in the dataset.

In [ ]:
# Load predictions
from scipy.io import loadmat

base_pred = loadmat('')
MNO_pred = loadmat('')